<a href="https://colab.research.google.com/github/anathayna/tcc/blob/main/notebooks/tcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color="orange">**TCC: Identificação do discurso de ódio em memes**</font>

O Google Colab é um serviço do Jupyter Notebook hospedado que oferece acesso gratuito a recursos de computação, incluindo GPUs e TPUs. Ele é adequado principalmente para aprendizado de máquina, ciência de dados e educação.

E este notebook documenta todo o desenvolvimento do trabalho de conclusão de curso que tem o objetivo de identificar o discurso de ódio em memes.

# <font color="orange">**Sumário**</font>

1.   Instalando as bibliotecas e dependências
2.   Adicionando o banco de dados
3.   Extração de características

Rascunho:
* Processamento das imagens
* Processamento dos textos
* Fine-tuning pre-treinamento
* Validação (fine-tuned)
* Gerando previsões
* Calculando probabilidades
* Classificação dos memes
* Avaliação (métricas)
* Conclusão

Preparar os dados: carregar train/val do dataset Hateful Memes.

Extrair embeddings com CLIP (imagem e texto).

Definir protótipos ou palavras-chave de referência para ativação fuzzy (e.g., “raça”, “religião”, imagens com símbolos ofensivos).

Calcular graus fuzzy separadamente para cada modalidade.

Criar regras fuzzy (talvez usando scikit-fuzzy ou outra biblioteca).

Inferir grau final de hatefulness e treinar ou calibrar thresholds com a base rotulada.

Avaliar com AUROC, F1, etc.


## <font color="orange">1. Instalando as bibliotecas e dependências</font>

Primeiro, configure o ambiente de execução do Google Colab para o GPU T4, que é voltado para aceleração de inferência de modelos de aprendizado profundo.

Outro detalhe que o Google Colab já vem com CUDA pré-instalado e drivers de GPU.

O **CUDA** (Compute Unified Device Architecture) é uma plataforma de computação paralela desenvolvida pela NVIDIA para acelerar processamentos usando GPUs (Graphics Processing Units). Ele permite que programas utilizem o poder de processamento massivamente paralelo das GPUs.

In [3]:
!nvcc --version  # Verifica se o CUDA está disponível
!nvidia-smi  # Mostra detalhes da GPU (como versão suportada)

/bin/bash: line 1: nvcc: command not found
/bin/bash: line 1: nvidia-smi: command not found


Próximo passo é conferir se o PyTorch 1.7.1 (ou uma versão superior) e o torchvision estão instalados.

In [4]:
import torch
import torchvision
import numpy as np
import torch
from pkg_resources import packaging

print("Versão do Torch:", torch.__version__)
print("Versão do Torchvision:", torchvision.__version__)

ModuleNotFoundError: No module named 'torchvision'

Após conferir as instalações iniciais, serão instaladas as seguintes dependências: o pacote *ftfy* (para normalização de texto), a biblioteca *regex* (processamento de strings) e *tqdm* (monitoramento de progresso).

In [ ]:
%pip install ftfy regex tqdm

Essas instalações são pré-requisito para a execução do modelo CLIP (Contrastive Language–Image Pretraining) da OpenAI, que será a metodologia central deste trabalho. Em seguida, será feito a instalação do CLIP por meio de sua implementação oficial disponível no GitHub.

In [ ]:
%pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-rou6mwig
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-rou6mwig
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


O CLIP fornece os seguintes modelos:

In [ ]:
import clip

clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

*   **ViT-B/32**: mais rápido e leve
*   **ViT-L/14**: tem maior precisão e alcança melhores resultados, mas exigem mais GPU
*   **ViT-B/16**: equilíbrio entre velocidade e acurácia



Tais modelos permitem analisar e extrair propriedades relevantes para seu uso, como o **ViT-B/32** que é o mais utilizado. Ele possui resolução de entrada de 224x224 pixels, contexto textual máximo de 77 tokens e vocabulário de cerca de 49 mil tokens.

A análise inclui o cálculo do total de parâmetros (aproximadamente 151 milhões), métrica que quantifica a complexidade e capacidade de aprendizado da rede.

Isso evidencia como avaliar escalabilidade e limitações operacionais de modelos multimodais, cuja operação demanda alto poder computacional e impõe restrições ao tamanho de textos e à resolução de imagens processáveis.

In [ ]:
model, preprocess = clip.load("ViT-B/32")
model.cuda().eval()

input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Total de parâmetros do modelo:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Resolução das imagens:", input_resolution) # ex: 224x224
print("Comprimento de contexto:", context_length) # número máximo de tokens
print("Tamanho do vocabulário:", vocab_size) # quantidade de palavras/tokens únicos

Total de parâmetros do modelo: 151,277,313
Resolução das imagens: 224
Comprimento de contexto: 77
Tamanho do vocabulário: 49408


## <font color="orange">2. Adicionando o banco de dados</font>

Serão utilizados dois bancos de dados para a identificação do discurso de ódio em memes.

O primeiro é o **Hateful Memes** disponibilizado pela Meta no *Challenge Hateful Memes*, um banco de dados com mais de **10 mil** imagens de memes em inglês, que contêm conteúdo ofensivo relacionado a gênero, raça, religião, orientação sexual, classe social e outros tópicos.

E o conjunto de dados é composto pelas seguintes porcentagens:

- 40% de memes de ódio multimodal (multimodal hate)
- 10% de memes de ódio unimodal (unimodal hate)
- 20% de memes com confusão de texto benigna (benign text confounder)
- 20% de memes com confusão de imagem benigna (benign image confounder)
- 10% de memes não odiosos aleatórios (random non-hateful)

Os **confundidores benignos** consistem em imagens ou textos alternativos para cada meme odioso, que alteram sua classificação para não odioso. Esses confundidores foram adicionados para garantir que o conjunto de dados seja apropriado para testar a verdadeira capacidade multimodal de um modelo, levando em conta que, no mundo real, o modelo se depararia com exemplos diversos e inéditos.

![](https://drivendata-public-assets.s3.amazonaws.com/memes-overview.png)

**Figura 1:** Exemplo de meme utilizado na competição  
Fonte: DRIVENDATA (2020)

**TO-DO:**
- MMHS150k dataset (multimodal hate speech)
- Memotion dataset (sentiment classification of memes)

Baixe o banco de dados do **Hateful Memes** pela plataforma *Kaggle* no seguinte endereço: https://www.kaggle.com/datasets/williamberrios/hateful-memes

Originalmente, esse conjunto de dados estava disponível no site oficial do desafio (https://hatefulmemeschallenge.com/#download). Entretanto, no momento da elaboração desta pesquisa, o domínio encontrava-se indisponível.

In [ ]:
#@markdown Defina o caminho para o arquivo **.zip** do banco de dados do *Hateful Memes*.
#@markdown **exemplo:** `"/content/drive/MyDrive/hateful_memes.zip"`

PATH_TO_ZIP_FILE = '/content/drive/MyDrive/hateful_memes.zip' #@param {type:"string"}

#@markdown Defina o diretório base para extrair o banco de dados.
#@markdown **exemplo:** `"/content"`

HOME = '/content' #@param {type:"string"}


Após realizar o download do banco de dados e armazená-lo no Google Drive, é necessário integrá-lo ao ambiente do Google Colab para extração das características. Para isso, utilizamos a biblioteca google.colab.drive, que permite montar o Google Drive como um sistema de arquivos virtual no ambiente de execução.

Ao executar o código, o Colab solicitará uma autorização para acessar sua conta do Google Drive. Uma vez concedida a permissão, o Drive será vinculado ao diretório `/content/drive/`, possibilitando a leitura dos arquivos armazenados. Essa abordagem garante acesso contínuo aos dados durante o processamento, sem necessidade de uploads manuais.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Agora, preparamos o ambiente para incorporar a pasta "model", que armazenará o banco de dados e outros arquivos essenciais, garantindo que o projeto possa acessá-los corretamente.

In [ ]:
import os
os.chdir(HOME)
os.getcwd()
os.environ['PYTHONPATH'] += ":/content/model/"

Em seguida, o código realiza a extração automática do banco de dados Hateful Memes (armazenado no formato `.zip`) para o diretório `/content/model/`, disponibilizando os arquivos para as próximas etapas de processamento e análise no notebook.

In [ ]:
import zipfile
zip_path = PATH_TO_ZIP_FILE
extract_path = '/content/model/'

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
img_dir = '/content/model/hateful_memes/img'

image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.webp']

image_count = 0
for filename in os.listdir(img_dir):
    if any(filename.lower().endswith(ext) for ext in image_extensions):
        image_count += 1

print(f"total de imagens: {image_count}")

total de imagens: 12140


## <font color="orange">3. Extração de características</font>

Extraia texto e imagem com os rótulos (0/1).
pré-processamento de texto: tokenização, deteção de insultos, análise de sentimento

crie variáveis fuzzy: por ex. InsultoText? com valores “baixo”, “médio”, “alto”; ImagemAgressiva? com níveis “nenhum”, “moderado”, “forte”.

regras
IF InsultoText é alto AND ImagemAgressiva é forte THEN GrauHate é alto
IF InsultoText é baixo AND ImagemAgressiva é baixo THEN GrauHate é baixo

fuzzificação, cálculo das regras, agregação e desfuzzificação para cada meme, gerando um grau contínuo de “ódio”

Converta o grau fuzzy em decisão: defina um limiar (ex: >= 0.5 → “hateful”, caso contrário “não-hateful”)

- load model
- prepare inputs: image & text
- calculate features: image & text encode = encoded features
- clip.tokenize
- model.encode_image
- model.encode_text

In [ ]:
%pip install pandas jsonlines

import os
import jsonlines
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
import torch

class HatefulMemesDataset(Dataset):
    def __init__(self, jsonl_path, img_dir, transform=None):
        """
        Args:
            jsonl_path (str): Caminho para o arquivo .jsonl (ex: "train.jsonl").
            img_dir (str): Caminho para a pasta com as imagens (ex: "img/").
            transform (callable, optional): Pré-processamento (ex: `preprocess` do CLIP).
        """
        self.img_dir = img_dir
        self.transform = transform

        # Lê o arquivo .jsonl e converte para DataFrame
        with jsonlines.open(jsonl_path) as reader:
            self.data = pd.DataFrame(reader)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx]["img"]
        # Correct the path by removing the 'img/' prefix from img_name
        # if it exists, before joining with the base image directory.
        if img_name.startswith("img/"):
             img_name = img_name[4:] # Remove 'img/'

        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        label = self.data.iloc[idx]["label"]

        if self.transform:
            image = self.transform(image)

        return image, label  # Retorna (imagem, rótulo)

base_path = '/content/model/hateful_memes/'
img_dir = os.path.join(base_path, "img")
train_jsonl = os.path.join(base_path, "train.jsonl")

import clip

# Carrega o modelo CLIP e o pré-processador
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device)

# Cria o dataset de treino
train_dataset = HatefulMemesDataset(
    jsonl_path=train_jsonl,
    img_dir=img_dir,
    transform=preprocess
)

# Exemplo: visualizar uma amostra
image, label = train_dataset[0]
print("Label:", label)  # 0 ou 1

# Conjuntos de desenvolvimento e teste
dev_seen_dataset = HatefulMemesDataset(
    jsonl_path=os.path.join(base_path, "dev_seen.jsonl"),
    img_dir=img_dir,
    transform=preprocess
)

test_unseen_dataset = HatefulMemesDataset(
    jsonl_path=os.path.join(base_path, "test_unseen.jsonl"),
    img_dir=img_dir,
    transform=preprocess
)

from torch.utils.data import DataLoader

batch_size = 32

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True  # Embaralha os dados para treino
)

dev_loader = DataLoader(
    dev_seen_dataset,
    batch_size=batch_size,
    shuffle=False  # Não embaralha para avaliação
)

Label: 0


# Task
Elaborar um plano de trabalho para a identificação de discurso de ódio em memes, incluindo a extração de características visuais e textuais, a incorporação de múltiplos datasets, o fine-tuning de um modelo pré-treinado (CLIP), a validação do modelo, a classificação dos memes, a análise dos resultados e a documentação do processo para um trabalho de conclusão de curso.

## Extração de características

### Subtask:
Processar as imagens e textos utilizando o modelo CLIP.


**Reasoning**:
Iterate through the data loaders to process images and texts using the CLIP model.



In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device) # Ensure model is on the correct device

def get_features(dataloader, dataset, model, device):
    all_image_features = []
    all_text_features = []
    all_labels = [] # Collect labels for train and dev sets

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            # Get the original text from the dataset using indices
            # Assuming the dataloader provides indices or the order is preserved
            # This part needs careful handling based on how the dataset and dataloader are implemented
            # For simplicity here, we'll assume we can get the text based on the batch
            # A more robust solution might involve modifying the dataset to return text along with image and label

            # Placeholder for text extraction - this needs to be adapted based on dataset structure
            # If the dataset returns indices, use them to get text from the dataframe
            # If not, and shuffle is True (train_loader), the order is not guaranteed
            # For now, we'll skip text encoding as the current dataset implementation doesn't provide text in __getitem__
            # We need to modify the HatefulMemesDataset to return text as well.

            # For now, let's just process images
            image_features = model.encode_image(images)
            all_image_features.append(image_features.cpu())
            all_labels.append(labels) # Collect labels

    return torch.cat(all_image_features), torch.cat(all_labels)

# Note: The current HatefulMemesDataset implementation does not return text.
# To process text with CLIP, the dataset needs to be modified to include text in its __getitem__ method.
# For now, we will only extract image features.

# Example usage (extracting image features):
train_image_features, train_labels = get_features(train_loader, train_dataset, model, device)
dev_image_features, dev_labels = get_features(dev_loader, dev_seen_dataset, model, device)

# For test_unseen, there are no labels to collect
def get_test_features(dataloader, dataset, model, device):
    all_image_features = []

    with torch.no_grad():
        for images, _ in dataloader: # Assuming dataloader still returns a second element (placeholder)
            images = images.to(device)
            image_features = model.encode_image(images)
            all_image_features.append(image_features.cpu())

    return torch.cat(all_image_features)

# Create a DataLoader for test_unseen_dataset
test_unseen_loader = DataLoader(
    test_unseen_dataset,
    batch_size=batch_size,
    shuffle=False
)

test_unseen_image_features = get_test_features(test_unseen_loader, test_unseen_dataset, model, device)


print("Train image features shape:", train_image_features.shape)
print("Train labels shape:", train_labels.shape)
print("Dev image features shape:", dev_image_features.shape)
print("Dev labels shape:", dev_labels.shape)
print("Test unseen image features shape:", test_unseen_image_features.shape)

# To process text, the HatefulMemesDataset needs to be updated to return text.
# Once the dataset is updated, the get_features function can be modified to encode text as well.

Train image features shape: torch.Size([8500, 512])
Train labels shape: torch.Size([8500])
Dev image features shape: torch.Size([500, 512])
Dev labels shape: torch.Size([500])
Test unseen image features shape: torch.Size([2000, 512])
